In [ ]:
#installs telebot and cv2 libraries using pip

#!pip install pyTelegramBotAPI
#https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
#use the below method to pip install telebot instead, refer to above link for explanation

# Install a pip package in the current Jupyter kernel
import sys
#another package called telebot which has been depracated, if installed wrong one then one way that worked was uninstalling telebot
#from the anaconda prompt (didn't seem to work in jupyter notebook)
#telebot!{sys.executable} -m pip install pyTelegramBotAPI
#cv2!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install ibm_watson

In [ ]:
#import libraries

#image processing library
import cv2
#telegram bot library
import telebot
#threading library, for multiple flows of code execution
import threading
#numpy library, a numerical computing python library
import numpy

from datetime import datetime
import json

In [ ]:
#stores formatted telebot messages in txt file
def archive_message(message) -> None:
    #see message contents
    print (message)

In [ ]:
#global variable declaration
image_counter = 0

In [ ]:
#starts telegram bot

#create telegram bot
bot = telebot.TeleBot("<insert Telebot API key here>")

In [ ]:
#default reply to message, echoes the text back
def default_reply(message) -> str:
    return message.text

In [ ]:
#message handlers

#if this is changed, then need to redeclare the bot object. I suspect it's because the new message handlers do not override
#the previous ones (and they are based on sequential order of declaration with the original message handlers declared first)

#start message handler, called when /start command is sent
@bot.message_handler(commands=['start'])
def send_welcome(message) -> None:
    #prints message to the console
    reply = "Hi, I'm a Telegram Bot. How are you?"
    bot.send_message(message.chat.id, reply)
    #print message text and reply
    print (message.text + "\t|\t" +  reply)
    
#demonstrate use of regex expression, all characters in string are integer characters
@bot.message_handler(regexp="[0-9]")
def all_numbers(message) -> None:
    reply = "That is a string of numbers"
    bot.reply_to(message, reply)
    print (message.text + "\t|\t" + reply)
    
#default message handler
@bot.message_handler(func=lambda message: True)
def echo_all(message) -> None:
    #default_reply echoes the message, later on Watson Assistant API is used to give chatbot functionality
    reply = default_reply(message)
    bot.send_message(message.chat.id, reply)
    print (message.text + "\t|\t" + reply)

In [ ]:
#start polling for messages

#if rerunning this kernel, need to stop for a few seconds to prevent simultaneous requests to the telegram API
bot.polling()

In [ ]:
#def get_image function to get image frame from camera
def get_image(camera_index:int=0) -> numpy.ndarray:
    #read image from camera_index (default 0, i.e. first camera)
    video = cv2.VideoCapture(0)
    ret, frame = video.read()
    return frame

In [ ]:
#get image and print image 

#gets image
frame = get_image()

#displays image
cv2.imshow("test", frame)

#closes image window when key is pressed
cv2.waitKey(0)

In [ ]:
#add message handler to take images
@bot.message_handler(commands=["photo"])
def take_photo(message) -> None:
    frame = get_image()
    #access global image_counter variable
    global image_counter
    #probably not the best solution but it keeps an archive of the images and sends it
    #need to find a better way to send the image directly as numpy array or without saving it
    #this makes it a little slow serving photo requests
    cv2.imwrite(f"temp_image{image_counter}.jpg", frame)
    #sends image
    bot.send_photo(chat_id=message.chat.id, photo=open(f"temp_image{image_counter}.jpg", "rb"))
    #increment image_counter so image is not overwritten (for archival)
    image_counter += 1

In [ ]:
#import IBM Watson libraries for IBM Assistant chatbot
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
#IBM Watson Assistant

#create Watson Assistant object
authenticator = IAMAuthenticator('<insert Watson Assistant API key here>')
assistant = AssistantV2(version='2019-02-28',authenticator=authenticator)

watson_assistant_id = "<insert Watson Assistant ID here>"

#function to create IBM Watson Assistant session using Watson Assistant object
def create_session() -> str:
    #return session_id
    session_id = assistant.create_session(assistant_id=watson_assistant_id).get_result()["session_id"]
    return session_id


#set service url for Watson Assistant, based on Assistant URL
assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com')

In [ ]:
#should check if file exists

try:
    #open up a file to check for existing session ids
    with open("session_id_list.json", "r") as infile:
        infile = infile.read().strip()
        session_id_dict = json.loads(infile) if infile != "" else {}
except IOError:
    #file does not exist
    session_id_dict = {}

In [ ]:
#setting up Watson API for default message handler
#sort of demonstrates polymorphism, with the initial default_reply having same name as this one
def default_reply(message) -> str:
    chat_id = message.chat.id
    now = datetime.now()
    if chat_id not in session_id_dict:
        #need to add a check for inactivity timeout, 5 minutes
        #before session id expires I think
        #need to create new session id
        session_id = create_session()
        session_id_dict[chat_id] = {"session_id": session_id, "time": now.strftime("%Y%m%d%H%M%S")}
    else:
        #for comparison
        now_compressed_int = int(now.strftime("%Y%m%d%H%M%S"))
        #get the last time chat was used
        last_chat_time = int(session_id_dict[chat_id]["time"])
        if now_compressed_int > last_chat_time + 500:
            #compare the strings and if greater than 500 means
            #5 minutes difference (can compare date time object as another method)
            #create a new session, can be split together with
            #above into a store session method
            session_id = create_session()
            session_id_dict[chat_id] = {"session_id": session_id, "time":str(now_compressed_int)}
        else:
            #reuse session id
            session_id = session_id_dict[chat_id]["session_id"]
            session_id_dict[chat_id]["time"] = str(now_compressed_int)
        
    #write new session_id_dict to file
    with open("session_id_list.json", "w") as outfile:
        outfile.write(json.dumps(session_id_dict))
    
    #get reply and return
    return assistant.message(\
                assistant_id = watson_assistant_id,\
                session_id=session_id,\
                input={"message_type": "text","text": message.text}\
                ).get_result()["output"]["generic"][0]["text"]

In [ ]:
#RPi GPIO

#import RPi GPIO library to access RPi's GPIO pins
import RPi.GPIO as GPIO